In [91]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import time
import re

In [126]:
driver = webdriver.Chrome('/Users/inika/Selenium Webdrivers/chromedriver')
driver.get('https://instagram.com')
# driver.maximize_window()

#login
sample_username = ""
sample_password = ""

wait = WebDriverWait(driver, 10)#driver, upper limit
login = wait.until(ec.presence_of_element_located((By.XPATH,"//button[@type = 'submit']")))
inputs = driver.find_elements_by_tag_name('input')
inputs[0].send_keys(sample_username)
inputs[1].send_keys(sample_password)
login.click()

time.sleep(2)
try:
    driver.find_element_by_xpath("//*[contains(text(), 'Not Now')]").click()
except NoSuchElementException:
    pass

## Question 1: 

Now your friend has followed a lot of different food bloggers, he needs to analyse the habits of these bloggers.
From the list of instagram handles you obtained when you searched ‘food’ in previous project. 
- Open the first 10 handles and find the top 5 which have the highest number of followers
- Now Find the number of posts these handles have done in the previous 3 days.
- Depict this information using a suitable graph.

In [127]:
searchbox = driver.find_element_by_xpath("//input[@placeholder = 'Search']") #search box
searchbox.send_keys("food")
time.sleep(1)
li = list(driver.find_elements_by_xpath("//div[@class = 'fuqBx']/*")) #list of items that show up
handles = []
for i in li:
    result = i.get_attribute('href')
    if 'location' not in result and 'tags' not in result:
        x = i.find_element_by_xpath('./div/div/div')
        handles.append(x.text)
        if len(handles)==10:
            break

clear = driver.find_element_by_xpath("//div[contains(@class, 'coreSpriteSearchClear')]")
clear.click()

followers = {}
for h in handles:
    searchbox = driver.find_element_by_xpath("//input[@placeholder = 'Search']") #search box
    searchbox.send_keys("@"+h)
    profile = wait.until(ec.presence_of_element_located((By.XPATH,"//div[@class = 'fuqBx']/a")))
    driver.get(profile.get_attribute('href'))
    followers[h] = int(re.sub("[^0-9]","",driver.find_element_by_xpath("//a[text() = ' followers']/span").get_attribute('title')))
    if len(followers)==10:
        break

In [129]:
# handles
print("Top 5 profiles with highest number of followers: ")
d = sorted(followers.items(),key=lambda x:x[1],reverse=True)[:5]
for i,t in enumerate(d):
    print("{}. {}: {}".format(i+1,t[0],t[1]))

Top 5 profiles with highest number of followers: 
0. foodnetwork: 10131031
1. yourfoodlab: 907814
2. dilsefoodie: 369544
3. foodtalkindia: 292617
4. food: 292617
